PROCESS DOCUMENT

In [ ]:
import os
from IPython.display import display, Markdown
from utils.document_processor import DocumentProcessor
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

## Inisialisasi pemrosesan dokumen
docs = DocumentProcessor()
DATA_PATH = "./data"
extracted_docs = []
valid_extensions = ('.pdf', '.docx', '.txt')

# **Baca dan Proses Semua Dokumen**
all_texts = []
metadata_list = []
file_texts = {}

# Baca dan proses dokumen
for filename in os.listdir(DATA_PATH):
    if not filename.lower().endswith(valid_extensions):
        continue  # Skip file yang tidak didukung

    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, "rb") as f:
        document = f.read()
        result = docs.process_document(document, filename)
        content, images, tables, plain_text = result

        # **Simpan teks per file**
        file_texts[filename] = plain_text
        
        # **Split teks menjadi chunk**
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        chunks = text_splitter.split_text(plain_text)

        # **Tambahkan metadata (nama file sebagai sumber)**
        for chunk in chunks:
            all_texts.append(chunk)
            metadata_list.append({"source": filename})

# Gunakan Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_texts(all_texts, embedding_model, metadatas=metadata_list)

retriever = vectorstore.as_retriever()

TEST OLLAMA MODEL

In [97]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [98]:
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vectore_test"

In [99]:
template = """
  You are a helpful assistant for text summarization. 
  Only include information that is part of the document. 
  Do not include your own opinion or analysis.

  Document: 
  "{document}"
  Summary:
"""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model=OLLAMA_MODEL)


RAG AGENTS

In [ ]:
# Retrieval QA Chain
retrieval_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

# Fungsi untuk Menampilkan Jawaban dengan Sumber
def retrieve_answer(query):
    response = retrieval_chain.invoke({"query": query})
    answer = response["result"]
    sources = {doc.metadata["source"] for doc in response["source_documents"]}
    
    source_text = "\n\n**Sumber:**\n" + "\n".join(sources)
    return answer + source_text

# Fungsi untuk Ringkasan per Dokumen
def summarize_document(doc_name):
    if doc_name in file_texts:
        summary_prompt = f"Ringkas dokumen berikut:\n\n{file_texts[doc_name]}\n\nRingkasan:"
        summary = model.invoke(summary_prompt)
        return f"**Ringkasan dari {doc_name}:**\n{summary}"
    else:
        return f"Dokumen '{doc_name}' tidak ditemukan dalam sistem."
    
# Fungsi untuk Ringkasan Semua Dokumen
def summarize_all_documents(_=None): 
    summaries = []
    for filename, text in file_texts.items():
        summary_prompt = f"Ringkas dokumen berikut:\n\n{text}\n\nRingkasan:"
        summary = model.invoke(summary_prompt)
        summaries.append(f"**Ringkasan dari {filename}:**\n{summary}")
    
    return "\n\n".join(summaries)

# Definisikan Tools untuk Agent
tools = [
    Tool(
        name="retrieval_qa",
        func=retrieve_answer,
        description="Gunakan ini untuk menjawab pertanyaan berdasarkan dokumen yang tersedia."
    ),
    Tool(
        name="summarize_document",
        func=summarize_document,
        description="Gunakan ini untuk meringkas satu dokumen. Masukkan nama file dokumen sebagai input."
    ),
    Tool(
        name="summarize_all_documents",
        func=summarize_all_documents,
        description="Gunakan ini untuk meringkas semua dokumen yang tersedia."
    )
]

In [ ]:
# Inisialisasi Agent dengan Multi-Query Handling
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# Coba Jalankan Agent dengan Pertanyaan
queries = [
    "Ringkas semua dokumen tentang kesehatan mental."
]

# Jalankan dan Tampilkan Hasil
for query in queries:
    print(f"\n### Pertanyaan: {query}\n")
    response = agent.invoke({"input": query})
    
    # Ambil final answer dari response
    final_answer = response.get("output", "Tidak ada jawaban yang dihasilkan.")
    
    # Tampilkan final answer dalam format Markdown
    display(Markdown(f"### Final Answer:\n\n{final_answer}"))


### Pertanyaan: Ringkas semua dokumen tentang kesehatan mental.



> Entering new AgentExecutor chain...
Thought: Aku harus menjawab pertanyaan ini dengan menggunakan alat "summarize_all_documents" untuk meringkas semua dokumen yang tersedia.

Action: summarize_all_documents
Action Input: None
Observation: **Ringkasan dari 22778-88667-1-PB.pdf:**
Artikel ini tidak memberikan ringkasan yang jelas tentang topik yang dibahas. Namun, berdasarkan judul dan beberapa kalimat yang saya baca, artikel ini mungkin membahas tentang pandemi COVID-19 dan dampaknya terhadap kesehatan mental di Indonesia.

Beberapa hal yang saya identifikasi sebagai topik yang dibahas adalah:

* Pandemi COVID-19 dan dampaknya terhadap kesehatan mental
* Kebijakan kesehatan mental di Indonesia dan tantangannya
* Dampak pandemi COVID-19 terhadap kebijakan kesehatan mental di Indonesia
* Pengaruh pandemi COVID-19 terhadap kesehatan mental masyarakat Indonesia

Namun, perlu diingat bahwa artikel ini tidak memberikan ring

### Final Answer:

Ringkasan dari semua dokumen tentang kesehatan mental di Indonesia adalah sebagai berikut:

Kesehatan mental di Indonesia masih memiliki beberapa tantangan, seperti stigma yang kuat terhadap penderita gangguan jiwa. Buku "Kesehatan Mental Masyarakat Indonesia" membahas pentingnya menjaga keseimbangan antara kehidupan fisik dan mental serta menghilangkan stigma yang memalukan bagi penderita gangguan jiwa.

Dalam beberapa tahun terakhir, pandemi COVID-19 telah menambah beban stres dan kecemasan pada masyarakat Indonesia, sehingga dampaknya terhadap kesehatan mental sangat signifikan. Penelitian "Analisis Kesehatan Mental Mahasiswa Perguruan Tinggi X Tahun Akaademik 2019/2020" menunjukkan bahwa mahasiswa masih mengalami stres, kecemasan dan depresi yang dapat mempengaruhi keseharaan dan kemampuan mereka.

Namun, ada beberapa upaya untuk meningkatkan kesadaran dan penanganan kesehatan mental di Indonesia. Buku "Kesehatan Mental Sumber Daya Manusia Indonesia" berisi daftar sumber dan judul penelitian yang terkait dengan topik kesehatan mental, serta beberapa paragraf yang membahas tentang pentingnya menjaga keseimbangan antara kehidupan fisik dan mental.

Dalam beberapa tahun terakhir, pemerintah Indonesia telah meningkatkan upaya untuk meningkatkan kesadaran dan penanganan kesehatan mental di masyarakat. Contohnya adalah program "Kesehatan Mental Masyarakat" yang diluncurkan oleh Kementerian Kesehatan Republik Indonesia, yang bertujuan untuk meningkatkan kesadaran dan penanganan kesehatan mental di masyarakat.

Ringkasan dari semua dokumen tersebut menunjukkan bahwa kesehatan mental di Indonesia masih memiliki beberapa tantangan, tetapi ada juga upaya untuk meningkatkan kesadaran dan penanganan kesehatan mental di masyarakat.